In [1]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [2]:
from transformers import TFAutoModel

bert = TFAutoModel.from_pretrained("bert-base-cased")

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [3]:
bert.summary()

Model: "tf_bert_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  108310272 
Total params: 108,310,272
Trainable params: 108,310,272
Non-trainable params: 0
_________________________________________________________________


In [4]:
import tensorflow as tf

## two inputs
#input_ids = tf.keras.layers.Input(shape=(512,), name="input_ids", dtype="int32")
#mask = tf.keras.layers.Input(shape=(512,), name="mask", dtype="int32")
#
## transformer
#embeddings = bert.bert(input_ids, attention_mask=mask)[1]
#
## classifier head
#x = tf.keras.layers.Dense(1024, activation="relu")(embeddings)
#outputs = tf.keras.layers.Dense(5, activation="softmax")(x)

# two input layers, we ensure layer name variables match to dictionary keys in TF dataset
input_ids = tf.keras.layers.Input(shape=(50,), name='input_ids', dtype='int32')
mask = tf.keras.layers.Input(shape=(50,), name='attention_mask', dtype='int32')

# we access the transformer model within our bert object using the bert attribute (eg bert.bert instead of bert)
embeddings = bert.bert(input_ids, attention_mask=mask)[1]  # access final activations (alread max-pooled) [1]
# convert bert embeddings into 5 output classes
x = tf.keras.layers.Dense(1024, activation='relu')(embeddings)
y = tf.keras.layers.Dense(5, activation='softmax', name='outputs')(x)

# model
model = tf.keras.Model(inputs=[input_ids, mask], outputs=y)

Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.


In [5]:
model.layers[2].trainable=False

In [6]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 50)]         0                                            
__________________________________________________________________________________________________
attention_mask (InputLayer)     [(None, 50)]         0                                            
__________________________________________________________________________________________________
bert (TFBertMainLayer)          TFBaseModelOutputWit 108310272   input_ids[0][0]                  
                                                                 attention_mask[0][0]             
__________________________________________________________________________________________________
dense (Dense)                   (None, 1024)         787456      bert[0][1]                   

In [7]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy("accuracy")

model.compile(optimizer=optimizer,
              loss=loss,
              metrics=[acc])

In [8]:
element_spec = ({'input_ids': tf.TensorSpec(shape=(16, 50), dtype=tf.int32, name=None),
  'attention_mask': tf.TensorSpec(shape=(16, 50), dtype=tf.int32, name=None)},
 tf.TensorSpec(shape=(16, 5), dtype=tf.float64, name=None))

In [9]:
train_ds = tf.data.experimental.load("train", element_spec=element_spec)
val_ds = tf.data.experimental.load("val", element_spec=element_spec)

In [10]:
train_ds.take(1)

<TakeDataset shapes: ({input_ids: (16, 50), attention_mask: (16, 50)}, (16, 5)), types: ({input_ids: tf.int32, attention_mask: tf.int32}, tf.float64)>

In [11]:
history = model.fit(train_ds,
                    validation_data=val_ds,
                    epochs=3)

Epoch 1/3
8778/8778 [==============================] - 4346s 494ms/step - loss: 1.1017 - accuracy: 0.5541 - val_loss: 1.0801 - val_accuracy: 0.5506
Epoch 2/3
8778/8778 [==============================] - 4643s 529ms/step - loss: 1.0435 - accuracy: 0.5737 - val_loss: 1.0281 - val_accuracy: 0.5715
Epoch 3/3
8778/8778 [==============================] - 4650s 530ms/step - loss: 1.0305 - accuracy: 0.5787 - val_loss: 1.0184 - val_accuracy: 0.5733
